In [1]:
using StatsBase
using JLD
using DataFrames
using Statistics

import IterTools

include("./model.jl")

run_model (generic function with 2 methods)

In [2]:
function get_block(true_planet)
    block =[]
    for p in true_planet
        if p < 20
            append!(block,1)
        elseif p < 40
            append!(block,2)
        elseif p < 60
            append!(block,3)
        elseif p < 80
            append!(block,4)
        elseif p < 100
            append!(block,5)
        end
    end
    return block
end


function chunk_galaries(galaxies)
    groups=[]
    for i in IterTools.groupby(x -> x, galaxies)
        push!(groups,i)
    end
        return groups
end

function number_planet_in_galaxy(galaxies)
    chunks = chunk_galaries(galaxies)
    return [i for galaxy in chunks for i in 1:length(galaxy)]
end

number_planet_in_galaxy (generic function with 1 method)

# Simulations

## Three Patch Environment 

In [6]:
function three_patch_inference(alpha)
    if alpha
        params = [1,5,2]
        filename = "model_results/show_inference_1.csv"
    else
        params = [0,5,2]
        filename = "model_results/show_inference_0.csv"
    end
    
    data = get_sub_data(76)
    num_particles = 1
    b,x,y,z=crp_adaptiveDiscount(data,params,num_particles)
    opt_prt = optimal_policy(b)
    diff = b.prt - opt_prt
    
    # put in dataframe and save
    df = DataFrame(Dict("true_planet"=> b.true_planet,"galaxy"=> b.galaxy,"prt"=>b.prt,
        "opt_prt"=>opt_prt, "diff" => diff))

    gdf = groupby(df, :galaxy)
    prt_avg = combine(gdf, :diff => mean)


    df = DataFrame(Dict("true_planet"=> b.true_planet,"galaxy"=> b.galaxy,"prt"=>b.prt,
        "opt_prt"=>opt_prt, "diff" => diff,"pred_decay"=>x,"uncertainty"=>y,"true_decay"=>z))
    
    CSV.write(filename,df)
    return  
end

three_patch_inference (generic function with 1 method)

In [ ]:
three_patch_inference(true)
three_patch_inference(false)

## Simulate models with best fitting parameters

In [3]:
function sub_params(model, sub_num, output, fit_params)
    sub_data = get_sub_data(sub_num)
    if cmp(model,"adaptive_discount") == 0
        if fit_params # load in the best fitting parameters 
            d=load(string("fit_params/adaptive_discount_10_10_3/sub",string(sub_num),".jld"))
            params = d["res"]
        else 
            params = [5,1,0.6]
        end
        num_particles = 1
        b = crp_adaptiveDiscount(sub_data,params,num_particles);
        df_params = DataFrame(Dict("sub_num"=> sub_num,"alpha"=> params[1],"gamma_base"=> params[2],"gamma_coef"=> params[3]))
    elseif cmp(model,"mvt") == 0
        if fit_params
            d=load(string("fit_params/MVT_learn_1_20_3/sub",string(sub_num),".jld"))
            params = d["res"]
        else
            params = [0.1,10,1]           
        end
        b = MVT_learn(sub_data,params);
        df_params = DataFrame(Dict("sub_num"=> sub_num,"alpha"=> params[1],"beta"=> params[2],"c"=> params[3]))

    elseif cmp(model,"td") == 0
        if fit_params
            d=load(string("fit_params/td_1_20_1_3/sub",string(sub_num),".jld"))
            params = d["res"]
        else
            params = [0.5,10,0.97,2]
        end
        b = TD(sub_data,params);
        df_params = DataFrame(Dict("sub_num"=> sub_num,"alpha"=> params[1],"beta"=> params[2],"gamma"=> params[3],"c"=> params[4]))

    end

    opt_prt = optimal_policy(b);
    diff = b.prt - opt_prt;
                
    df = DataFrame(Dict("true_planet"=> b.true_planet,"galaxy"=> b.galaxy,"block"=> get_block(b.true_planet),"prt"=>b.prt,
        "opt_prt"=>opt_prt, "diff" => diff))
    df[!,"n_galaxy"] = number_planet_in_galaxy(df.galaxy)
       
    
    if cmp(output,"avg_and_params") == 0
        gdf = groupby(df, :galaxy)
        prt_avg = combine(gdf, :diff => mean)
        insertcols!(prt_avg,       # DataFrame to be changed
        1,                # insert as column 1
        :sub_num => ones(size(prt_avg)[1])*sub_num,   # populate as "Day" with 1,2,3,..
        makeunique=true) 
    
        insertcols!(df_params,       # DataFrame to be changed
        1,                # insert as column 1
        :sub_num => ones(size(df_params)[1])*sub_num,   # populate as "Day" with 1,2,3,..
        makeunique=true) 
        
        return prt_avg, df_params
    elseif cmp(output,"full") == 0
        insertcols!(df,       # DataFrame to be changed
        1,                # insert as column 1
        :sub_num => ones(size(df)[1])*sub_num,   # populate as "Day" with 1,2,3,..
        makeunique=true) 

        return df 
    end
end

function all_subs(model,subs,output,fit_params)
    if cmp(output,"avg_and_params") == 0
        df_prt = DataFrame()
        df_params = DataFrame()
        for sub in subs
            try
                prt, params = sub_params(model,sub,output,fit_params)
                df_prt = vcat(df_prt,prt)
                df_params = vcat(df_params,params)
            catch err
            end
        end
        return df_prt, df_params
    elseif cmp(output,"full") == 0
        df_prt = DataFrame()
        for sub in subs
            for n_sim=1:50
                try
                    prt = sub_params(model,sub,output,fit_params)
                    insertcols!(prt,       # DataFrame to be changed
                        1,                # insert as column 1
                    :n_sim => ones(size(prt)[1])*n_sim,   # populate as "Day" with 1,2,3,..
                    makeunique=true) 
                    df_prt = vcat(df_prt,prt)
                catch err
                end
            end
        end
        return df_prt
    end
end 

all_subs (generic function with 1 method)

In [11]:
subs=[  0,   1,   2,   3,   5,   8,   9,  10,  13,  15,  18,  19,  21,
        22,  23,  25,  28,  29,  30,  31,  32,  33,  34,  37,  39,  40,
        41,  44,  45,  46,  47,  48,  50,  53,  54,  55,  56,  57,  58,
        59,  64,  65,  69,  70,  71,  75,  76,  77,  78,  80,  81,  82,
        85,  89,  92,  94,  96,  97,  99, 100, 101, 104, 105, 106, 107,
       108, 110, 112, 113, 115, 116, 117, 119, 120, 121, 123, 124, 126,
       127, 128, 132, 134, 135, 136, 137, 138, 141, 142, 143, 146, 148,
       151, 154, 158, 159, 161, 162, 163, 164, 165, 167, 168, 169, 170,
       173, 175, 177, 182, 183, 184, 188, 190, 192, 195, 196, 197]

prt,params = all_subs("adaptive_discount",subs,"avg_and_params",true)
CSV.write("model_results/prt_val_adaptive_discount_10_10_3.csv",prt)
CSV.write("model_results/params_val_adaptive_discount_10_10_3.csv",params)

prt,params = all_subs("mvt",subs,"avg_and_params",true)
CSV.write("model_results/prt_val_MVT_learn.csv",prt)
CSV.write("model_results/params_val_MVT_learn.csv",params)

prt,params = all_subs("td",subs,"avg_and_params",true)
CSV.write("model_results/prt_val_TD.csv",prt)
CSV.write("model_results/params_val_TD.csv",params)

"model_results/params_val_TD.csv"

In [5]:
# only want subs with alpha > 1 and gamma_coef > 0 for adaptive discounting model
params = DataFrame(CSV.File("model_results/params_val_adaptive_discount_10_10_3.csv"))
select_subs=filter(row -> row.alpha >= 0.8 && row.gamma_coef>0.00, params).sub_num

84-element Array{Int64,1}:
   1
   2
   9
  10
  15
  18
  19
  21
  25
  28
  31
  33
  37
   ⋮
 169
 170
 173
 175
 182
 183
 184
 188
 190
 195
 196
 197

In [4]:
subs=[  0,   1,   2,   3,   5,   8,   9,  10,  13,  15,  18,  19,  21,
        22,  23,  25,  28,  29,  30,  31,  32,  33,  34,  37,  39,  40,
        41,  44,  45,  46,  47,  48,  50,  53,  54,  55,  56,  57,  58,
        59,  64,  65,  69,  70,  71,  75,  76,  77,  78,  80,  81,  82,
        85,  89,  92,  94,  96,  97,  99, 100, 101, 104, 105, 106, 107,
       108, 110, 112, 113, 115, 116, 117, 119, 120, 121, 123, 124, 126,
       127, 128, 132, 134, 135, 136, 137, 138, 141, 142, 143, 146, 148,
       151, 154, 158, 159, 161, 162, 163, 164, 165, 167, 168, 169, 170,
       173, 175, 177, 182, 183, 184, 188, 190, 192, 195, 196, 197]


prt = all_subs("adaptive_discount",subs,"full",true)
CSV.write("model_results/overharvest_adapt_adaptive_discount_10_10_3.csv",prt)

prt = all_subs("td",subs,"full",true)
CSV.write("model_results/overharvest_adapt_TD.csv",prt)

prt = all_subs("mvt",subs,"full",true)
CSV.write("model_results/overharvest_adapt_MVT.csv",prt)

"model_results/overharvest_adapt_MVT.csv"

In [38]:
prt = all_subs("adaptive_discount",subs,"full",false)
CSV.write("model_results/overharvest_adapt_adaptive_discount_opt_params.csv",prt)

prt = all_subs("td",subs,"full",false)
CSV.write("model_results/overharvest_adapt_TD_opt_params.csv",prt)

"model_results/overharvest_adapt_TD_opt_params.csv"